In [ ]:
## Standard :

# Name imports :
import os
import gc
import copy
import sklearn
import itertools
import time
import typing
import random

# Aliased imports :
import multiprocessing as mp
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sb
import matplotlib.pyplot as plt

# Partial imports :
from scipy.integrate import quad as integrate
from sklearn.neighbors import KernelDensity
from sklearn.svm import SVR
from sklearn import preprocessing

# Full imports :
from plotnine import *a

In [4]:
def select_date_range(df: pd.core.frame.DataFrame, 
                     start_date: str, 
                     end_date: str) -> pd.core.frame.DataFrame:
    """
    """
    mask = (df.index >= start_date) & (df.index <= end_date)
    
    

def time_indexed_df(df1: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    """ Cast into a time-indexed dataframe.
    df1 paramater should contain a column called 'dateTime',
    which contains entries of type pandas._libs.tslibs.timestamps.Timestamp
    """
    _tmp = copy.deepcopy(df1)
    _tmp.index = df1.dateTime
    _tmp.drop('dateTime', axis=1, inplace=True)
    _tmp = _tmp.sort_index()
    gc.collect()
    return _tmp

def merge_date_time(df1: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    """ Take a dataframe containing columns:
        'Date'
        'Time'
        
        And return one containing a single column:
         'dateTime' = 'Date' + 'Time'
        
        For each entry as seen below:
         '2019-03-21 17:34:05' <- '2019-03-21' + '17:34:05'
    """
    _tmp = copy.deepcopy(df1)
    _tmp['dateTime'] = _tmp['Date'] + ' ' + _tmp['Time']
    _tmp.drop(['Date', 'Time'], axis=1, inplace=True)
    gc.collect()
    return _tmp

def hybrid_interpolator(data: pd.core.series.Series,
                        mean: float = None,
                        limit: float = None,
                        methods: typing.List[str] = ['linear', 'spline'], 
                        weights: typing.List[float] = [0.65, 0.35],
                        direction: str = 'forward',
                        order: int = 2
                       ) -> pd.core.series.Series:
    """
    Return a pandas.core.series.Series instance resulting of the weighted average
    of two interpolation methods.
    
    Model:
        φ = β1*method1 + β2*method2
        
    Default:
        β1, β2 = 0.6, 0.4
        method1, method2 = linear, spline
    
    Weights are meant to be numbers from the interval (0, 1)
    which add up to one, to keep the weighted sum consistent.
    
    limit_direction : {‘forward’, ‘backward’, ‘both’}, default ‘forward’
    If limit is specified, consecutive NaNs will be filled in this direction.
    
    If the predicted φ_i value is outside of the the interval
    ( (mean - limit), (mean + limit) )
    it will be replaced by the linear interpolation approximation.
    
    If not set, mean and limit will default to:
        mean = data.mean()
        limit = 2 * data.std()
    
    This function should have support for keyword arguments, but is yet to be implemented.
    """
    predictions: typing.List[float] = [] 
    
    if not np.isclose(sum(weight for weight in weights), 1):
        raise Exception('Sum of weights must be equal to one!')
    
    for met in methods:
        if (met == 'spline') or (met == 'polynomial'):
            predictions.append(data.interpolate(method=met, order=order, limit_direction=direction))
        else:
            predictions.append(data.interpolate(method=met, limit_direction=direction))

    linear: pd.core.series.Series = predictions[0]
    spline: pd.core.series.Series = predictions[1]
    hybrid: pd.core.series.Series = weights[0]*predictions[0] + weights[1]*predictions[1]
    
    corrected: pd.core.series.Series = copy.deepcopy(hybrid) 
    
    if not mean:
        mean = data.mean()
    if not limit:
        limit = 2 * data.std()
    
    for idx, val in zip(hybrid[ np.isnan(data) ].index, hybrid[ np.isnan(data) ]):
        if (val > mean + limit) or (val < mean - limit):
            corrected[idx] = linear[idx]
    
    #df = copy.deepcopy(interpolated)
    #print(df.isnull().astype(int).groupby(df.notnull().astype(int).cumsum()).sum())
    
    return corrected
    
    


In [21]:
#def predict_prices(dates, prices, x):
#    dates = np.reshape(dates,(len(dates), 1)) # convert to 1xn dimension
#    x = np.reshape(x,(len(x), 1))

class SVRegressor(object):
    
    def __init__(self, 
                 C: float = 1e3, 
                 degree: int = 2, 
                 gamma: float = 0.1,
                 X: np.ndarray = None,
                 y: np.ndarray = None,
                 features: str = 'X',
                 labels: str = 'y'
                ):
        self._keys = ['linear', 'poly', 'rbf']   
        self._svrs = {
            'linear': SVR(kernel='linear', C=C), 
            'poly': SVR(kernel='poly', C=C, degree=degree), 
            'rbf': SVR(kernel='rbf', C=C, gamma=gamma)
        }
        self._labels = {
            'features': features, 
            'labels': labels
        }
        if X is not None and y is not None:
            if type(X) is np.ndarray and type(y) is np.ndarray:
                self._X = X
                self._y = y
            else:
                raise Exception('type() X and y should be numpy.ndarray')
        else:
            self._X = X
            self._y = y
    ##
    
    def __getitem__(self, key):
        if key in self.keys:
            return self._svrs[key]
        else:
            raise Exception(f'{key} not found in keys. Possible values are: {self.keys}')
    ##
    
    @property
    def keys(self):
        return self._keys
    ##
    
    @property
    def kernels(self):
        return self._keys
    ##
    
    def set_training_data(self, X, y):
        if type(X) is np.ndarray and type(y) is np.ndarray:
            self._X = X
            self._y = y
        else:
            raise Exception('type() X and y should be numpy.ndarray')
    ##
    
    @property
    def training_data(self):
        ''' Returns self._X (features), self._y (labels)'''
        return self._X, self._y
    ##
    
    def fit(self, kernel: str = 'all'):
        if kernel == 'all':
            [ self._svrs[i].fit(self._X, self._y) for i in self.keys ]
        elif kernel in self.kernels:
            self._svrs[kernel].fit(self._X, self._y)
        else:
            raise Exception(f'Invalid kernel, available kernels are {self.kernels}')
    ##
    
    def metrics(self, X: np.ndarray, y: np.ndarray, kernel: str = 'all'):
        if kernel == 'all':
            predictions = { 
                kernel: self._svrs[kernel].predict(X) for kernel in self.kernels
            }
        elif kernel in self.kernels:
            prediction = self._svrs[kernel].predict(X)
            corr       = np.corrcoef(prediction, y)[0][1]
            _devs      = y - prediction
            _abs_devs  = np.abs(_devs)
        else:
            raise Exception(f'Invalid kernel, available kernels are {self.kernels}')
    ##
    
    def plot(self, X: np.ndarray = None, y: np.ndarray = None, 
             kernel: str = 'all', xlabel: str = 'X', ylabel: str = 'y'):
        
        if X is not None and y is not None:
            if type(X) is not np.ndarray or type(y) is not np.ndarray:
                raise Exception('Input type() for X and y shoud be numpy.ndarray')
            elif X.shape[1] != self._X.shape[1]:
                message = f'Model was trained on a {self._X.shape[1]}-dimensional space, input is {X.shape[1]}-dimensional'
                raise Exception(message)
            elif X.shape[0] != y.shape[0]:
                raise Exception('Features (X), and labels (y) must contain the same number of observations.')
            else:
                X = preprocessing.scale(X)
        else:
            X = self._X
            y = self._y
        
        if kernel == 'all':
            if len(X.shape) == 1:
                plt.scatter(X, y, c='k', label='Data')
                for key, color in zip(self._svrs.keys(), ['g', 'r', 'b']):
                    plt.plot(X, self._svrs[key].predict(X), c=color, label=key)
            else:
                _dummy_x = [i for i in range(len(y))]
                plt.scatter(_dummy_x, y, c='k', label='Data')
                for i, key, color in zip(range(3), self._svrs.keys(), ['g', 'r', 'b']):
                    plt.plot(_dummy_x, self._svrs[key].predict(X), c=color, label=key)
        
        elif kernel in self.kernels:
            if len(X.shape) == 1:
                plt.scatter(X, y, c='k', label='Data')
                plt.plot(X, self._svrs[kernel].predict(X), c='g', label=kernel)
            else:
                _dummy_x = [i for i in range(len(y))]
                plt.scatter(_dummy_x, y, c='k', label='Data')
                plt.plot(_dummy_x, self._svrs[kernel].predict(X), c='g', label=kernel)
        
        else:
            raise Exception(f'Invalid kernel, available kernels are {self.kernels}')
        
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.title('Support Vector Regression')
        plt.legend()
        plt.show()
    ##
    
    def normalize_features(self):
        if type(self._X) is not None:
            self._X = preprocessing.scale(X)
        else:
            raise Exception('Training data not set.')
    ##
    
    def predict(self, kernel: str = 'all',  X: np.ndarray = None):
        """ Acutal predictions are the first element, to access them:
            SVRegressor.predict()[0]
        """
        if not X:
            X = self._X
        elif type(X) is not np.ndarray:
            raise Exception('Input type(X), shoud be numpy.ndarray')
        else:
            X = preprocessing.scale(X)
        
        if kernel == 'all':
            return  { 
                kernel: self._svrs[kern].predict(X) for kern in self.kernels 
            }
        elif kernel not in self.kernels:
                raise Exception(f'Kernel not found. Possible values are: all, {self.kernels}')
        else:
            return self._svrs[kernel].predict(X) 
    ##

In [5]:
raw = pd.read_csv('data/carelink2.csv')
raw = merge_date_time(raw)

In [6]:
pool = mp.Pool() # processes parameter can be set manually, 
                 # but this is suposed to spawn the same number as the system has cores.

raw.dateTime = pool.map(pd.to_datetime, raw.dateTime)

pool.close()
pool.terminate()

In [7]:
undesired_columns = [
    'Index',
    'Temp Basal Type', 
    'Temp Basal Duration (h:mm:ss)',
    'BWZ Target High BG (mg/dL)', 
    'BWZ Target Low BG (mg/dL)',
    'Bolus Type',
    'Insulin Action Curve Time',
    'New Device Time',
    'Bolus Duration (h:mm:ss)',
    'Prime Type', 
    'Prime Volume Delivered (U)',
    'Alarm',
    'ISIG Value',
    'Event Marker',
    'Bolus Number',
    'Suspend', 
    'Rewind',
    'Linked BG Meter ID',
    'Bolus Cancellation Reason',
    'Scroll Step Size',
    'Sensor Calibration Rejected Reason',
    'Network Device Associated Reason',
    'Network Device Disassociated Reason',
    'Network Device Disconnected Reason',
    'Sensor Exception',
    'Preset Temp Basal Name',
    'Preset Bolus', 
    'Bolus Source'
]

In [8]:
raw = raw.drop(undesired_columns, axis=1)
unsure_columns = [
    'BG Reading (mg/dL)',
    'Sensor Calibration BG (mg/dL)'
]

In [9]:
#
data = pd.read_csv('binaries/first_half_resampled.csv', encoding="utf-8-sig")
len(data.index)

46860

In [10]:
#data.columns
data = data.set_index('grouper')
data.head()

,Glucose (t),Glucose (t+1),Glucose (t+2),Glucose (t+3)
grouper,,,,
2019-01-01 00:00:00,241.0,325.0,268.0,257.0
2019-01-01 00:01:00,243.8,325.0,266.0,257.0
2019-01-01 00:02:00,246.6,325.0,264.0,257.0
2019-01-01 00:03:00,249.4,325.0,262.0,257.0
2019-01-01 00:04:00,252.2,325.0,260.0,257.0


In [11]:
data['Slope 1min'] = data['Glucose (t+3)'].diff(periods=1).rolling(window=1).mean()
data['Mean Slope 5min'] = data['Glucose (t+3)'].diff(periods=1).rolling(window=5).mean()
data['Total Slope 5min'] = data['Glucose (t+3)'].diff(periods=1).rolling(window=5).sum()
data['Slope Std. Dev,'] = data['Glucose (t+3)'].diff(periods=1).rolling(window=5).std()
data['Max slope'] = data['Glucose (t+3)'].diff(periods=1).rolling(window=5).max()
data['Min slope'] = data['Glucose (t+3)'].diff(periods=1).rolling(window=5).min()
data['Glucose (t+3:15)'] = data.shift(-15)['Glucose (t+3)']

In [12]:
data.head(17)

,Glucose (t),Glucose (t+1),Glucose (t+2),Glucose (t+3),Slope 1min,Mean Slope 5min,Total Slope 5min,"Slope Std. Dev,",Max slope,Min slope,Glucose (t+3:15)
grouper,,,,,,,,,,,
2019-01-01 00:00:00,241.0,325.0,268.0,257.0,NaN,NaN,NaN,NaN,NaN,NaN,241.0
2019-01-01 00:01:00,243.8,325.0,266.0,257.0,0.0,NaN,NaN,NaN,NaN,NaN,232.0
2019-01-01 00:02:00,246.6,325.0,264.0,257.0,0.0,NaN,NaN,NaN,NaN,NaN,223.0
2019-01-01 00:03:00,249.4,325.0,262.0,257.0,0.0,NaN,NaN,NaN,NaN,NaN,214.0
2019-01-01 00:04:00,252.2,325.0,260.0,257.0,0.0,NaN,NaN,NaN,NaN,NaN,205.0
2019-01-01 00:05:00,255.0,325.0,258.0,257.0,0.0,0.00,0.0,0.000000,0.0,0.0,196.0
2019-01-01 00:06:00,256.4,325.2,258.0,255.4,-1.6,-0.32,-1.6,0.715542,0.0,-1.6,193.0
2019-01-01 00:07:00,257.8,325.4,258.0,253.8,-1.6,-0.64,-3.2,0.876356,0.0,-1.6,190.0
2019-01-01 00:08:00,259.2,325.6,258.0,252.2,-1.6,-0.96,-4.8,0.876356,0.0,-1.6,187.0


In [13]:
slicer = data.columns[0:-1]
data = data.reset_index().drop('grouper', axis=1)
data = data.dropna()

In [15]:
data.shape


(46840, 11)

In [16]:
data.head()

,Glucose (t),Glucose (t+1),Glucose (t+2),Glucose (t+3),Slope 1min,Mean Slope 5min,Total Slope 5min,"Slope Std. Dev,",Max slope,Min slope,Glucose (t+3:15)
5,255.0,325.0,258.0,257.0,0.0,0.00,0.0,0.000000,0.0,0.0,196.0
6,256.4,325.2,258.0,255.4,-1.6,-0.32,-1.6,0.715542,0.0,-1.6,193.0
7,257.8,325.4,258.0,253.8,-1.6,-0.64,-3.2,0.876356,0.0,-1.6,190.0
8,259.2,325.6,258.0,252.2,-1.6,-0.96,-4.8,0.876356,0.0,-1.6,187.0
9,260.6,325.8,258.0,250.6,-1.6,-1.28,-6.4,0.715542,0.0,-1.6,184.0


In [17]:
X = data.loc[1:30240, slicer[0]:slicer[-1]].values
y = data.loc[1:30240, data.columns[-1]].values

In [18]:
X.shape

(30236, 10)

In [19]:
X2 = data.loc[30240:35000:, slicer[0]:slicer[1]].values
y2 = data.loc[30240:35000, data.columns[-1]].values

In [24]:
R = SVRegressor()
R.set_training_data(X, y)
R.normalize_features()

In [25]:
### To measure execution time (this method might not be very accurate, use with precaution)

R.fit(kernel='rbf')

print(f'{elapsed - start}')

/home/adfuurt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.


TypeError: unsupported operand type(s) for -: 'float' and 'builtin_function_or_method'

In [ ]:
R.plot_training(kernel='rbf')

In [ ]:
R.plot_test(X=X2, y=y2, kernel='rbf')